# SQL

Цель исследования — проанализировать базу данных с информацией о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

# 1. Импорт библиотек и изучение данных

In [1]:
import pandas as pd
from sqlalchemy import create_engine

Импортируем необходимые библиотеки.

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
 'pwd': 'Sdf4$2;d-d30pp', # пароль
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, # порт подключения
 'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

Напишем код подключения к базе данных.

In [3]:
def select_top(table, n):
    """Функция возвращает первые n строк таблицы table"""
    query = ''' SELECT *
                FROM {0}
                LIMIT {1}
            '''.format(table, n)
    return pd.io.sql.read_sql(query, con = engine) 

select_top('books', 5)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Изучим таблицы базы данных - выведем по 5 первых строк каждой таблицы.

In [4]:
select_top('authors', 5)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
select_top('publishers', 5)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [6]:
select_top('ratings', 5)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [7]:
select_top('reviews', 5)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


# 2. Исследовательский анализ данных

## 2.1. Число книг после 1 января 2000 года

In [8]:
query = ''' SELECT COUNT(book_id)
            FROM books
            WHERE publication_date > '2000-01-01'
        '''
pd.io.sql.read_sql(query, con = engine)

,count
0,819


После 1 января 2000 года вышло 819 книг.

## 2.2. Количество обзоров и средняя оценка для каждой книги

In [9]:
query = ''' SELECT books_reviews.title, COUNT(DISTINCT review_id) AS reviews_cnt, AVG(rating) AS avg_rating
            FROM (SELECT books.book_id, reviews.review_id, books.title
                  FROM books LEFT JOIN reviews 
                  ON books.book_id = reviews.book_id) AS books_reviews LEFT JOIN ratings
                  ON ratings.book_id = books_reviews.book_id
            GROUP BY books_reviews.book_id, books_reviews.title
            ORDER BY reviews_cnt DESC, avg_rating DESC
        '''
pd.io.sql.read_sql(query, con = engine).head(10)

,title,reviews_cnt,avg_rating
0,Twilight (Twilight #1),7,3.662500
1,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
2,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
3,The Book Thief,6,4.264151
4,The Glass Castle,6,4.206897
5,Outlander (Outlander #1),6,4.125000
6,The Hobbit or There and Back Again,6,4.125000
7,The Curious Incident of the Dog in the Night-Time,6,4.081081
8,The Lightning Thief (Percy Jackson and the Oly...,6,4.080645
9,Water for Elephants,6,3.977273


Для каждой книги запишем количество обзоров в столбец reviews_cnt и среднюю оценку в столбец avg_rating. Отсортируем по убыванию числа ревью и среднего рейтинга и выведем первые 10 строк таблицы. Больше всего ревью у первой части саги Сумерки и Гарри Поттера (части Узник Азкабана и Тайная комната), средний рейтинг Сумерек ниже других из топ-5.

## 2.3. Издательство с наибольшим числом книг толще 50 страниц

In [10]:
query = '''         SELECT publishers.publisher, COUNT(book_id) AS books_cnt
                    FROM books LEFT JOIN publishers 
                    ON books.publisher_id = publishers.publisher_id
                    WHERE num_pages > 50
                    GROUP BY publishers.publisher_id  
                    HAVING COUNT(book_id) >= ALL(
                        SELECT COUNT(book_id) AS books_cnt
                        FROM books LEFT JOIN publishers 
                        ON books.publisher_id = publishers.publisher_id
                        WHERE num_pages > 50
                        GROUP BY publishers.publisher_id
                        )
        '''
pd.io.sql.read_sql(query, con = engine).head()

,publisher,books_cnt
0,Penguin Books,42


Penguin Books - издательство, которое выпустило наибольшее число книг (42 книги) толще 50 страниц.

## 2.4. Автор с самой высокой средней оценкой книг (для книг с 50 и более оценками)

In [11]:
query = '''SELECT author, AVG(rating) AS avg_rating
            FROM (
                SELECT books.book_id, authors.author_id, authors.author
                FROM books LEFT JOIN authors
                ON books.author_id = authors.author_id
            ) AS books_authors
            LEFT JOIN ratings 
            ON ratings.book_id = books_authors.book_id
            GROUP BY author_id, author
            HAVING AVG(rating) >= ALL(
                SELECT AVG(rating) AS avg_rating
                FROM (
                    SELECT books.book_id, authors.author_id, authors.author
                    FROM books LEFT JOIN authors
                    ON books.author_id = authors.author_id
                ) AS books_authors
                LEFT JOIN ratings 
                ON ratings.book_id = books_authors.book_id
                GROUP BY author_id, author
                HAVING COUNT(rating) > 50
            )
            AND COUNT(rating) > 50
            '''
pd.io.sql.read_sql(query, con = engine)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.288462


J.K. Rowling/Mary GrandPré - автор с самой высокой средней оценкой книг (4,29) среди книг с 50 и более оценками.

## 2.5. Среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [12]:
query = '''SELECT AVG(reviews_cnt) AS avg_reviews_cnt
           FROM (
               SELECT COUNT(review_id) AS reviews_cnt
               FROM reviews
               GROUP BY username
               HAVING username IN (
                    SELECT ratings.username
                    FROM ratings
                    GROUP BY username
                    HAVING COUNT(rating) > 50 
               )
            ) AS reviews_cnt_table
        '''
pd.io.sql.read_sql(query, con = engine)

,avg_reviews_cnt
0,24.333333


Cреднее количество обзоров от пользователей, которые поставили больше 50 оценок, примерно равно 24 обзорам.

# Итоговые выводы

Импортированы необходимые библиотеки, выполнено подключение к бд, изучены таблицы базы данных.

- После 1 января 2000 года вышло 819 книг.
- Для каждой книги количество обзоров записано в столбец reviews_cnt, а средняя оценка в столбец avg_rating
- Penguin Books - издательство, которое выпустило наибольшее число книг (42 книги) толще 50 страниц.
- J.K. Rowling/Mary GrandPré - автор с самой высокой средней оценкой книг (4,29) среди книг с 50 и более оценками.
- Cреднее количество обзоров от пользователей, которые поставили больше 50 оценок, примерно равно 24 обзорам.